In [1]:
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("ai4bharat/IndicCorpV2", "indiccorp_v2", split="hin_Deva",streaming=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

IterableDataset({
    features: Unknown,
    num_shards: 3
})

In [5]:
for i, example in enumerate(dataset):
    print(example)
    if i == 4:
        break

{'text': 'लोगों को बिलों संबंधी सुविधा देना ही उनका काम'}
{'text': ''}
{'text': 'इनेलो 1987 में उस वक्त ऐसे ही दोराहे पर खड़ी थी, जब पूर्व उपप्रधानमंत्री देवीलाल ने अपने पुत्र ओमप्रकाश चौटाला को अपना राजनीतिक उत्तराधिकारी घोषित किया था। हालांकि तब पार्टी पर देवीलाल की मजबूत पकड़ के चलते पार्टी टूटने से बच गई थी। 1989 में देवीलाल केन्द्र की राजनीति में सक्रिय हो गए थे और उनके उपप्रधानमंत्री बनने के पश्चात् उनके तीन बेटों जगदीश सिंह, रणजीत सिंह और ओमप्रकाश चौटाला में से रणजीत और ओमप्रकाश के बीच हरियाणा में उनकी राजनीतिक विरासत को लेकर जंग शुरू हो गई थी। उन परिस्थितियों में देवीलाल ने कड़ा निर्णय लेते हुए पार्टी की बागडोर ओमप्रकाश चौटाला के हवाले कर दी थी, जिसके बाद रणजीत की बगावत का असर पार्टी, संगठन और उनकी सरकार पर भी पड़ा था। उस समय रणजीत की नाराजगी के चलते उनके समर्थन में कई कैबिनेट मंत्रियों ने इस्तीफे दे दिए थे किन्तु तब पार्टी सुप्रीमो चौ. देवीलाल की हरियाणा की जनता पर इतनी मजबूत पकड़ थी कि ओमप्रकाश चौटाला को उत्तराधिकारी बनाने के उनके फैसले का जनता के बीच कोई खास विरोध नहीं हुआ थ

In [6]:
def sentence_tokenizer(text):
    sentence_endings = ['।', '?', '!', '.']
    sentences = []
    buffer = ''
    for char in text:
        buffer += char
        if char in sentence_endings:
            sentences.append(buffer.strip())
            buffer = ''
    if buffer.strip():  # trailing part
        sentences.append(buffer.strip())
    return sentences


In [7]:
import re

def find_special_tokens(text):
    special_tokens = []
    url_pattern = r'((?:https?://|www\.)[^\s,;:()"\']+)'
    email_pattern = r'([\w\.-]+@[\w\.-]+\.\w+)'
    emoji_pattern = r'([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F900-\U0001F9FF\U00002600-\U000026FF\U00002700-\U000027BF])'

    matches = []
    for p in [url_pattern, email_pattern, emoji_pattern]:
        for m in re.finditer(p, text):
            matches.append((m.start(), m.end(), m.group()))
    matches.sort()
    return matches

def word_tokenizer(sentence):
    specials = find_special_tokens(sentence)
    tokens = []
    last_idx = 0
    for start, end, val in specials:
        if start > last_idx:
            part = sentence[last_idx:start]
            tokens.extend(custom_split(part))
        tokens.append(val)
        last_idx = end
    if last_idx < len(sentence):
        tokens.extend(custom_split(sentence[last_idx:]))
    return [tok for tok in tokens if tok.strip()]

def custom_split(part):
    tokens = []
    token = ''
    for char in part:
        if char.isspace() or char in '.,?!;:"\'()[]{}|\\/।':
            if token:
                tokens.append(token)
                token = ''
            if char.strip():
                tokens.append(char)
        else:
            token += char
    if token:
        tokens.append(token)
    return tokens


In [8]:
output_file = "tokenized_hindi.txt"
with open(output_file, "w", encoding="utf-8") as fout:
    for row in tqdm(dataset, desc="Tokenizing dataset"):
        text = row['text']
        # Sentence tokenize
        sentences = sentence_tokenizer(text)
        for sent in sentences:
            tokens = word_tokenizer(sent)
            fout.write(" ".join(tokens) + "\n")

Tokenizing dataset: 141801328it [3:00:30, 13092.26it/s]


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data

In [9]:
def split_file_by_size(input_path, output_path, max_bytes):
    read_bytes = 0
    chunk_size = 1024 * 1024  # 1MB
    with open(input_path, 'rb') as fin, open(output_path, 'wb') as fout:
        while read_bytes < max_bytes:
            to_read = min(chunk_size, max_bytes - read_bytes)
            data = fin.read(to_read)
            if not data:
                break
            fout.write(data)
            read_bytes += len(data)

# Usage example:
GB = 1024 ** 3
split_file_by_size('tokenized_hindi.txt', 'tokenized_hindi_10gb.txt', max_bytes=10 * GB)
